# GUI

In [ ]:
# Microglia Tracking GUI
import itk
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import Rbf
from scipy.io import savemat, loadmat
import tifffile
import cv2
import random
from utils import *
from setGUIFont import *
import datetime

from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
from PyQt5.QtCore import *
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure

import cellpose.core
import cellpose.utils
import cellpose.io
import cellpose.models
import cellpose.metrics
import cellpose.denoise

# Elastix Config
class ElastixConfigWindow(QDialog):
    def __init__(self, parent, function):
        super().__init__(parent)
        self.parent = parent
        self.function = function
        self.setWindowTitle("Elastix Config")
        self.setGeometry(100, 100, 900, 600)
        self.dict_label = {}
        self.dict_entry = {}
        # Elastix parameter
        self.dict_parameter_map = parent.dict_parameter_map[function]
        
        layout = QGridLayout()
        # 4列になるようlabel, entryを配置
        for i, (key_parameter, tuple_value_parameter) in enumerate(self.dict_parameter_map.items()):
            row = i % 7
            column = i // 7
            self.dict_label[key_parameter] = QLabel(key_parameter)
            layout.addWidget(self.dict_label[key_parameter], row*2, column)
            value_parameter = ' '.join(tuple_value_parameter) # tupleをstrに変換
            self.dict_entry[key_parameter] = QLineEdit(value_parameter)
            layout.addWidget(self.dict_entry[key_parameter], row*2+1, column)
        
        self.setLayout(layout)
    
    def reject(self):
        # Elastix parameter 更新
        for key_parameter in self.dict_parameter_map.keys():
            value = list(self.dict_entry[key_parameter].text().split(" "))
            self.dict_parameter_map[key_parameter] = value
        self.parent.dict_parameter_map[self.function] = self.dict_parameter_map
        super().reject()
        self.close()
        
# CellPose Config
class CellposeConfigWindow(QDialog):
    def __init__(self, parent):
        super().__init__(parent)
        self.parent = parent
        self.setWindowTitle("Cellpose Config")
        self.setGeometry(100, 100, 400, 300)
        self.dict_label = {}
        self.dict_entry = {}
        
        # Cellpose parameter と その型、初期値
        self.dict_parameter = parent.dict_parameter_cellpose
        
        self.dict_parameter_types = {
            "gpu": bool,
            "model_type": str,
            "restore_type": str,
            "diameter": int,
            "channels": list
        }
        
        layout = QGridLayout()
        
        for i, (key, value) in enumerate(self.dict_parameter.items()):
            self.dict_label[key] = QLabel(key)
            layout.addWidget(self.dict_label[key], i, 0)
            
            self.dict_entry[key] = QLineEdit(str(value))
            layout.addWidget(self.dict_entry[key], i, 1)
        
        self.setLayout(layout)
    
    def reject(self):
        # Cellpose parameter 更新
        for key, value_type in self.dict_parameter_types.items():
            value = self.dict_entry[key].text()
            if value_type == bool:
                self.dict_parameter[key] = value.lower() == "true"
            elif value_type == int:
                self.dict_parameter[key] = int(value)
            elif value_type == list:
                self.dict_parameter[key] = eval(value)
            else:  # str などその他の型
                self.dict_parameter[key] = value_type(value)
        
        self.parent.dict_parameter_cellpose = self.dict_parameter
        super().reject()
        self.close()

class MicrogliaTracking(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("MicrogliaTrackingGUI")
        self.setGeometry(100, 100, 1200, 200)
        self.setupUI_done = False

        self.dict_label       = {} # labelの保管
        self.dict_entry       = {} # entryの保管
        self.dict_button      = {} # buttonの保管
        self.dict_ax          = {} # axの保管
        self.dict_checkbox    = {} # checkboxの保管
        self.dict_slider      = {} # sliderの保管
        self.dict_combobox    = {} # pulldownの保管
        self.dict_list        = {} # ListWidgetの保管
        self.dict_buttongroup = {}
        self.dict_scene       = {}
        self.dict_view        = {}
        self.dict_table       = {}
        self.dict_figure      = {}
        self.dict_canvas      = {}
        self.dict_opacity     = {}
        self.dict_roiPixmapItem = {}
        self.dict_roiPixmapItemHighlight = {}
        self.dict_table_selectedRow = {}
    
        self.setupFileLoadUI()
        
        self.dict_tif_original   = {}
        self.dict_tif_reg        = {}
        self.dict_grid_original  = {}
        self.dict_grid_reg       = {}
        self.dict_cellpose_stack = {} # Cellposeをかけた結果
        
        self.grid_square = 64 # 格子画像の格子の大きさ
        self.ROIOpacity_init = 50 # ROI透明度の初期値
        self.ROIcolors = self.makeCellposeROIColors(999)  # 999個のROI色を生成
        # Tableの列名と列番号のdict
        self.dict_celltypes = {"Microglia": 2, "Not Cell": 3, "Check": 4, "Tracking": 5, "Memo": 6}
        # Elastix parameter
        self.dict_parameter_map = {}
        for function in ["affine", "bspline"]:
            parameter_object = itk.ParameterObject.New()
            self.dict_parameter_map[function] = dict(parameter_object.GetDefaultParameterMap(function, 4).items())
        # Cellpose parameter
        self.dict_parameter_cellpose = {
            "gpu": True,
            "model_type": "cyto3",
            "restore_type": "denoise_cyto3",
            "diameter": 20,
            "channels": [0, 0]
        }
        
    # Widget, LayoutなどUI設定用の関数
    """
    UI setup Function
    """    
    # File load用のUIセットアップ
    def setupFileLoadUI(self):
        self.mainLayout = QGridLayout()
        
        # checkroi_fix, setup, checkroi_movのLayoutを配置するためのLayout
        self.Layout_setupcheckroi = QHBoxLayout()
        
        Layout_setup = QVBoxLayout()
        # ファイルパスとbrowseボタン
        layout_path_tif_pri = self.makeLayoutLoadFileWidget(label="Tiff image stack file path (Primary channel)", key="path_tif_pri", fileFilter="tiff Files (*.tif *.tiff);;All Files (*)")
        layout_path_tif_ref = self.makeLayoutLoadFileWidget(label="Tiff image stack file path (Reference channel)", key="path_tif_ref", fileFilter="tiff Files (*.tif *.tiff);;All Files (*)")
        
        self.Layout_setup_button = QHBoxLayout()
        # Loadボタンの設定
        self.dict_button["loadFile"] = QPushButton("Load files")
        self.dict_button["loadFile"].clicked.connect(self.loadFilePathsandInitialize)
        self.Layout_setup_button.addWidget(self.dict_button["loadFile"])
        # Exitボタンの設定
        self.dict_button["exit"] = QPushButton("Exit")
        self.dict_button["exit"].clicked.connect(self.exitApp)
        self.Layout_setup_button.addWidget(self.dict_button["exit"])
        
        Layout_setup.addLayout(layout_path_tif_pri)
        Layout_setup.addLayout(layout_path_tif_ref)
        Layout_setup.addLayout(self.Layout_setup_button)
        Layout_setup.setSpacing(0)  # レイアウト間のスペーシングを設定
        
        self.Layout_setupcheckroi.addLayout(Layout_setup)
        
        self.mainLayout.addLayout(self.Layout_setupcheckroi, 2, 0, 1, 2)
        
        widget = QWidget()
        widget.setLayout(self.mainLayout)
        self.setCentralWidget(widget)
    
    # GUI, データの初期化
    def initializeGUI(self):
        # tif読み込み
        for key_tif in ["pri", "ref"]:
            self.dict_tif_original[key_tif] = self.convertImageToINT(tifffile.imread(self.dict_entry[f"path_tif_{key_tif}"].text()))
            self.dict_tif_reg[key_tif] = self.dict_tif_original[key_tif].copy()
        # 格子画像の作成
        grid_coords = self.makeGridCoords(width=self.dict_tif_original["ref"].shape[2], 
                                          height=self.dict_tif_original["ref"].shape[1],
                                          square=self.grid_square)
        grid_image = self.convertGridCoordsToImage(grid_coords, width=self.dict_tif_original["ref"].shape[2],
                                                   height=self.dict_tif_original["ref"].shape[1])

        # 格子画像をスタックとして保存
        self.dict_grid_original = np.stack([grid_image] * self.dict_tif_original["ref"].shape[0], axis=0)
        self.dict_grid_reg = self.dict_grid_original.copy()
            
    # スタックスライダーの初期化関数を追加
    def initializeStackSliders(self):
        stack_size = len(self.dict_tif_original["ref"])
        for key in ["fix", "mov"]:
            self.dict_slider[f"stack_{key}"].setMaximum(stack_size - 1)
            self.dict_slider[f"stack_{key}"].setValue(0)
            self.updateStackIndexLabel(key, 0)
        
    # File load後のUIセットアップ
    def setupUI(self):
        self.setGeometry(100, 100, 1800, 1000)
        
        # Control Layout
        Layout_control = QHBoxLayout()
        Layout_control.addLayout(self.makeLayoutROICheckbuttonSlider("fix"))
        Layout_control.addLayout(self.makeLayoutROICheckbuttonSlider("mov"))
        
        # ROIMatch Layout
        Layout_roimatch = self.makeLayoutROIMatchingControl()
        Layout_control.insertLayout(1, Layout_roimatch)
        
        # ROI, Plot Layout
        Layout_fixmov = QHBoxLayout()
        Layout_fix = self.makeLayoutView("fix")
        Layout_mov = self.makeLayoutView("mov")
        Layout_fixmov.addLayout(Layout_fix)
        Layout_fixmov.addLayout(Layout_mov)
        
        # Table Layout
        Layout_tableroimatch = QVBoxLayout()
        Layout_table = self.makeLayoutMatchingTable()
        Layout_tableroimatch.addLayout(Layout_table)
        # ROI Matching config
        Layout_config = self.makeLayoutMatchingConfig()
        Layout_tableroimatch.addLayout(Layout_config)
        
        # Widget, Layoutの配置
        self.mainLayout.addLayout(Layout_fixmov, 0, 0, 1, 1)
        self.mainLayout.addLayout(Layout_control, 1, 0, 1, 1)
        self.mainLayout.addLayout(Layout_tableroimatch, 0, 1, 2, 1)

        self.setLayout(self.mainLayout)
        self.setupUI_done = True
        
    """
    make Layout Function
    """
    # 読み込むファイルを選択するためのウィジェット
    def makeLayoutLoadFileWidget(self, label="", key="", fileFilter=""):
        hboxlayout = QHBoxLayout() # entry, button
        vboxlayout = QVBoxLayout() # label, entry. button
        
        self.dict_label[key] = QLabel(label)
        self.dict_entry[key] = QLineEdit()
        self.dict_entry[key].setMinimumWidth(800)
        self.dict_button[f"browse_{key}"] = QPushButton("Browse")
        self.dict_button[f"browse_{key}"].clicked.connect(lambda: self.openFileDialog(fileFilter, self.dict_entry[key]))
        hboxlayout.addWidget(self.dict_entry[key])
        hboxlayout.addWidget(self.dict_button[f"browse_{key}"])
        vboxlayout.addWidget(self.dict_label[key])
        vboxlayout.addLayout(hboxlayout)
        return vboxlayout
    
    # Fixed, Moving imageのtable, view, plot WidgetをまとめたLayout
    def makeLayoutView(self, key): # key: fix, mov
        # Fixed Image Layout
        Layout = QVBoxLayout()
        
        Layout_Scene = QHBoxLayout()
        
        # ROI表示画像
        self.dict_scene[key] = QGraphicsScene()
        self.dict_view[key] = QGraphicsView(self.dict_scene[key])
        self.dict_view[key].setMinimumHeight(530)
        self.dict_view[key].setMinimumWidth(530)
        self.dict_view[key].setStyleSheet("background-color: black;")  # 背景色を黒に設定
        # クリック時のイベント
        self.dict_view[key].mousePressEvent = lambda event: self.viewMousePressEvent(event, key)
        # ROI, tif imageの設定
        self.dict_roiPixmapItem[key], self.dict_roiPixmapItemHighlight[key] = None, None
        
        Layout_Scene.addWidget(self.dict_view[key])
        
        Layout_Slider = QVBoxLayout()
        # スタックを動かすスライダー
        self.dict_slider[f"stack_{key}"] = QSlider(Qt.Horizontal)
        self.dict_slider[f"stack_{key}"].setMinimum(0)
        self.dict_slider[f"stack_{key}"].setMaximumHeight(15)
        self.dict_slider[f"stack_{key}"].valueChanged.connect(lambda value, k=key: self.updateStackDisplay(k, value))
        
        # Stack index label
        self.dict_label[f"stack_index_{key}"] = QLabel(f"Stack Index: 0")
        
        Layout_Slider.addWidget(self.dict_slider[f"stack_{key}"])
        Layout_Slider.addWidget(self.dict_label[f"stack_index_{key}"])
        
        Layout.addLayout(Layout_Scene)
        Layout.addLayout(Layout_Slider)
        return Layout
    
    # mov cellpose ROIのmatching用table Layout
    def makeLayoutMatchingTable(self):
        Layout_matching = QVBoxLayout()
        
        # TableViewの作成
        self.dict_table["mov_match"] = QTableView()
        self.dict_table["mov_match"].setMinimumWidth(200)  # 最小幅を設定
        
        # テーブルの初期化（スタック数が決まっていない場合は後で更新する）
        self.initializeMatchingTable()
        
        # クリックイベントを接続
        self.dict_table["mov_match"].clicked.connect(self.onTableClicked)
        
        Layout_matching.addWidget(self.dict_table["mov_match"])
        return Layout_matching
    
    # ROI Matchingのconfig Layout
    def makeLayoutMatchingConfig(self):
        Layout_matching = QHBoxLayout()
        
        # 選択したROIのstat
        Layout_ROI_stat = QVBoxLayout()
        for key, label in zip(["roi_cellid", "roi_stat_med", "roi_stat_npix"], ["ROI 0", "med: [0, 0]", "npix: 0"]):
            self.dict_label[key] = QLabel(label)
            Layout_ROI_stat.addWidget(self.dict_label[key])
        
        # ROI Matchingのthreshold
        Layout_matching_threshold = QVBoxLayout()        
        Layout_matching_threshold.addWidget(QLabel("ROI Matching Threshold"))
        Layout_matching_threshold_param = QGridLayout()
        for i, (key_label, value_entry) in enumerate(zip(["r", "error_rate"], ["5", "20%"])):
            self.dict_label[f"roimatch_threshold_{key_label}"] = QLabel(key_label)
            self.dict_entry[f"roimatch_threshold_{key_label}"] = QLineEdit(key_label)
            self.dict_entry[f"roimatch_threshold_{key_label}"].setText(value_entry)
            Layout_matching_threshold_param.addWidget(self.dict_label[f"roimatch_threshold_{key_label}"], 0, i, 1, 1)
            Layout_matching_threshold_param.addWidget(self.dict_entry[f"roimatch_threshold_{key_label}"], 1, i, 1, 1)
        Layout_matching_threshold.addLayout(Layout_matching_threshold_param)
        
        # ROI Matching button
        Layout_matching_button = QVBoxLayout()
        self.dict_button["roimatch_run"] = QPushButton("ROI Matching")
        self.dict_button["roimatch_run"].clicked.connect(self.runROIMatching)
        Layout_matching_button.addWidget(self.dict_button["roimatch_run"])
        
        Layout_matching.addLayout(Layout_ROI_stat)
        Layout_matching.addLayout(Layout_matching_threshold)
        Layout_matching.addLayout(Layout_matching_button)
        
        return Layout_matching
    
    # ROIの表示調整用Layout
    def makeLayoutROICheckbuttonSlider(self, key):
        Layout_ROIshow = QVBoxLayout()
        
        # チェックボックスの設定
        # fix, movで分ける
        if key == "fix":
            list_channel = ["pri", "ref", "grid"]
            list_func = [lambda state, key=key: self.toggleImageVisibility(key, state, "pri"),
                         lambda state, key=key: self.toggleImageVisibility(key, state, "ref"), 
                         lambda state, key=key: self.toggleImageVisibility(key, state, "grid")]
        elif key == "mov":
            list_channel = ["pri", "ref", "grid", "mask"]
            list_func = [lambda state, key=key: self.toggleImageVisibility(key, state, "pri"),
                         lambda state, key=key: self.toggleImageVisibility(key, state, "ref"), 
                         lambda state, key=key: self.toggleImageVisibility(key, state, "grid"),
                         lambda state, key=key: self.toggleImageVisibility(key, state, "mask")]
        Layout_ROIshow_checkbox = QHBoxLayout()
        for channel, func_ in zip(list_channel, list_func):
            self.dict_checkbox[f"show_{key}_{channel}"] = QCheckBox(f"Show {key} {channel} Image")
            self.dict_checkbox[f"show_{key}_{channel}"].setChecked(True)
            self.dict_checkbox[f"show_{key}_{channel}"].stateChanged.connect(func_)
            Layout_ROIshow_checkbox.addWidget(self.dict_checkbox[f"show_{key}_{channel}"])
            
        Layout_ROIshow.addLayout(Layout_ROIshow_checkbox)
        
        # Min, Max, Opacity Valueスライダーの設定
        Layout_ROIshow_sliderminmax = QHBoxLayout()
        Layout_ROIshow_slideropacity = QVBoxLayout()
        list_key_slider = ["pri", "ref"]

        # Min, Maxスライダー
        for key_slider in list_key_slider:
            Layout_ROIshow_sliderminmax_keyslider = QVBoxLayout()
            for m, default_value in zip(["min", "max"], [0, 255]):
                self.dict_slider[f"{m}Value_{key}_{key_slider}"] = QSlider(Qt.Horizontal)
                self.dict_slider[f"{m}Value_{key}_{key_slider}"].setMinimum(0)
                self.dict_slider[f"{m}Value_{key}_{key_slider}"].setMaximum(255)
                self.dict_slider[f"{m}Value_{key}_{key_slider}"].setMaximumHeight(5)
                self.dict_slider[f"{m}Value_{key}_{key_slider}"].setValue(default_value)
                self.dict_slider[f"{m}Value_{key}_{key_slider}"].valueChanged.connect(lambda value, k=key: self.updateImage(k, self.dict_slider[f"stack_{k}"].value()))
                self.dict_label[f"{m}Value_{key}_{key_slider}"] = QLabel(f"{m} Value ({key_slider})")
                Layout_ROIshow_sliderminmax_keyslider.addWidget(self.dict_label[f"{m}Value_{key}_{key_slider}"])
                Layout_ROIshow_sliderminmax_keyslider.addWidget(self.dict_slider[f"{m}Value_{key}_{key_slider}"])
            Layout_ROIshow_sliderminmax.addLayout(Layout_ROIshow_sliderminmax_keyslider)
            
        self.dict_slider[f"opacityValue_{key}"] = QSlider(Qt.Horizontal)
        self.dict_slider[f"opacityValue_{key}"].setMinimum(0)
        self.dict_slider[f"opacityValue_{key}"].setMaximum(255)
        self.dict_slider[f"opacityValue_{key}"].setMaximumHeight(5)
        self.dict_slider[f"opacityValue_{key}"].setValue(50)
        self.dict_slider[f"opacityValue_{key}"].valueChanged.connect(lambda value, key=key: self.updateMaskOpacity(key, value))
        self.dict_label[f"opacityValue_{key}"] = QLabel(f"opacity Value ({key})")
        Layout_ROIshow_slideropacity.addWidget(self.dict_label[f"opacityValue_{key}"])
        Layout_ROIshow_slideropacity.addWidget(self.dict_slider[f"opacityValue_{key}"])
        
        Layout_ROIshow.addLayout(Layout_ROIshow_sliderminmax)
        Layout_ROIshow.addLayout(Layout_ROIshow_slideropacity)
        
        return Layout_ROIshow
    
    # ROI Matching用のLayout
    def makeLayoutROIMatchingControl(self):
        Layout_roimatch = QHBoxLayout()
        
        Layout_roimatch_method = QVBoxLayout()
        # Registration用のMethod
        self.dict_label["regist_method"] = QLabel("Registration Method")
        # Elastixのconfig
        self.dict_button["config_elastix"] = QPushButton("Elastix config")
        self.dict_button["config_elastix"].clicked.connect(lambda: self.openElastixConfigWindow())
        self.dict_combobox["regist_method"] = QComboBox()
        self.dict_combobox["regist_method"].addItems(['affine','bspline'])
        self.dict_button["regist_method"] = QPushButton("Registration")
        self.dict_button["regist_method"].clicked.connect(lambda: self.runElastix())
        Layout_roimatch_method.addWidget(self.dict_label["regist_method"])
        Layout_roimatch_method.addWidget(self.dict_button["config_elastix"])
        Layout_roimatch_method.addWidget(self.dict_combobox["regist_method"])
        Layout_roimatch_method.addWidget(self.dict_button["regist_method"])
        
        # CellPose
        Layout_roimatch_cellpose = QVBoxLayout()
        self.dict_label["roimatch_cellpose"] = QLabel("CellPose")
        # Elastixのconfig
        self.dict_button["config_cellpose"] = QPushButton("CellPose config")
        self.dict_button["config_cellpose"].clicked.connect(lambda: self.openCellposeConfigWindow())
        self.dict_button["roimatch_cellpose"] = QPushButton("Run CellPose")
        self.dict_button["roimatch_cellpose"].clicked.connect(lambda: self.runCellpose())
        Layout_roimatch_cellpose.addWidget(self.dict_label["roimatch_cellpose"])
        Layout_roimatch_cellpose.addWidget(self.dict_button["config_cellpose"])
        Layout_roimatch_cellpose.addWidget(self.dict_button["roimatch_cellpose"])
        
        Layout_roimatch.addLayout(Layout_roimatch_method)
        Layout_roimatch.addLayout(Layout_roimatch_cellpose)
        return Layout_roimatch
            
    """
    Table Widget Function
    """
    # roi marching tableの初期化
    def initializeMatchingTable(self):
        if "mov_match" not in self.dict_table:
            return

        stack_size = len(self.dict_tif_original["ref"])
        column_count = stack_size * 2 - 1
        row_count = 100  # 仮の行数（必要に応じて調整）

        # モデルの作成
        model = QStandardItemModel(row_count, column_count)
        self.dict_table["mov_match"].setModel(model)

        # ヘッダーの設定
        horizontal_header = self.dict_table["mov_match"].horizontalHeader()
        horizontal_header.setSectionResizeMode(QHeaderView.Stretch)
        horizontal_header.setVisible(True)

        # 新しいヘッダーラベルを設定
        header_labels = []
        for i in range(1, stack_size):
            header_labels.extend([str(i-1), str(i)])
        header_labels.extend([str(i)])

        model.setHorizontalHeaderLabels(header_labels)

        # 垂直ヘッダーの設定
        vertical_header = self.dict_table["mov_match"].verticalHeader()
        vertical_header.setVisible(True)

        # 奇数列のセルの背景色を薄赤色に設定
        light_red = QColor(255, 200, 200)  # 薄赤色
        for col in range(0, column_count, 2):
            for row in range(row_count):
                item = QStandardItem()
                item.setBackground(light_red)
                model.setItem(row, col, item)

        self.dict_table["mov_match"].setModel(model)
        
    # テーブルのセル選択時の関数
    def onTableClicked(self, index):
        row = index.row()
        col = index.column()

        # 偶数列を選んだ場合は反応しない
        if col % 2 == 1:
            return

        current_stack_index = col // 2
        self.updateStackDisplay("mov", current_stack_index)

            
    # Viewで選択したROIをテーブルでも選択
    def selectTableCell(self, stack_index, roi_index):
        if "mov_match" not in self.dict_table:
            return

        table = self.dict_table["mov_match"]
        model = table.model()

        column_index = stack_index * 2

        cell_index = model.index(roi_index, column_index)
        table.setCurrentIndex(cell_index)
        table.scrollTo(cell_index)

        self.updateStackDisplay("mov", stack_index)
        
        
    # Cellposeで抽出したROIの番号を割り当てる(奇数列, roi matchのfix側)
    def assignROIsToTable(self):
        if "mov_match" not in self.dict_table:
            return

        model = self.dict_table["mov_match"].model()
        if model is None:
            return

        # 各画像のROI数を取得
        roi_counts = [mask.max() for mask in self.dict_cellpose_stack['mask']]

        # 必要な行数を計算（最大のROI数）
        max_roi_count = max(roi_counts)

        # 必要に応じて行数を増やす
        while model.rowCount() < max_roi_count:
            model.insertRow(model.rowCount())

        # ROIの番号をテーブルに割り当てる
        light_red = QColor(255, 200, 200)  # 薄赤色
        for i, roi_count in enumerate(roi_counts):
            col = i * 2  # 奇数列
            for row in range(roi_count):
                item = QStandardItem(str(row))
                item.setBackground(light_red)
                model.setItem(row, col, item)

        # 不要な行を削除
        while model.rowCount() > max_roi_count:
            model.removeRow(model.rowCount() - 1)

        self.dict_table["mov_match"].setModel(model)
    
    # キーイベントのカスタム
    """
    keyPressEvent
    """

            
    # UI上のクリック、スライダーの操作等に紐づける関数
    """
    UI-Event Function
    """
    # チャンネルの表示/非表示を切り替える
    def toggleImageVisibility(self, key, state, channel):
        self.updateStackDisplay(key, self.dict_slider[f"stack_{key}"].value())
            
    """
    Slider Function
    """


    # スタック表示を更新する
    def updateStackDisplay(self, key, index):
        self.dict_slider[f"stack_{key}"].setValue(index)
        self.updateImage(key, index)
        self.updateStackIndexLabel(key, index)

    # 画像を更新する
    def updateImage(self, key, index):
        # 1. Base image creation (pri, ref, grid)
        channels = ["ref", "pri", "grid"]
        images = []
        for channel in channels:
            if self.dict_checkbox[f"show_{key}_{channel}"].isChecked():
                img = self.getChannelImage(key, channel, index)
                if channel in ["pri", "ref"]:
                    img = self.adjustChannelContrast(key, channel, img)
                images.append(img)
            else:
                images.append(np.zeros_like(self.getChannelImage(key, channel, index)))

        rgb_image = np.stack(images, axis=-1)
        
        highlighted_roi = None  # 変数を初期化

        # 2. Overlay mask if available and checkbox is checked
        if key == "mov" and 'mask' in self.dict_cellpose_stack and index < len(self.dict_cellpose_stack['mask']):
            if self.dict_checkbox[f"show_{key}_mask"].isChecked():
                mask = self.dict_cellpose_stack['mask'][index]
                opacity = self.dict_slider["opacityValue_mov"].value() / 255.0

                # 3. Highlight selected ROI if applicable
                highlighted_roi = None
                if "mov_match" in self.dict_table:
                    model = self.dict_table["mov_match"].model()
                    current_index = self.dict_table["mov_match"].currentIndex()
                    if current_index.isValid():
                        current_item = model.item(current_index.row(), index * 2)
                        if current_item and current_item.text():
                            highlighted_roi = int(current_item.text())

                rgb_image = self.overlayMask(rgb_image, mask, opacity, highlighted_roi)

        # 4. Draw outline of corresponding ROI in next stack if applicable
        if key == "mov" and "mov_match" in self.dict_table and 'mask' in self.dict_cellpose_stack and index < len(self.dict_cellpose_stack['mask']) - 1:
            model = self.dict_table["mov_match"].model()
            current_index = self.dict_table["mov_match"].currentIndex()
            if current_index.isValid():
                next_item = model.item(current_index.row(), index * 2 + 1)
                if next_item and next_item.text() and next_item.text() != "-1":
                    next_roi_index = int(next_item.text())
                    outline = self.dict_cellpose_stack['outline'][index + 1][next_roi_index]
                    color = self.ROIcolors[next_roi_index % len(self.ROIcolors)]
                    rgb_image = self.drawOutline(rgb_image, outline, color)

        self.displayImage(key, rgb_image)

        # Update ROI stat if a ROI is highlighted
        if highlighted_roi is not None:
            self.updateROIStat(index, highlighted_roi)
        
    # overlayMask メソッド
    def overlayMask(self, image, mask, opacity, highlighted_roi=None):
        mask_rgb = np.zeros((*mask.shape, 3), dtype=np.uint8)
        for i in range(1, mask.max() + 1):
            color = self.ROIcolors[(i-1) % len(self.ROIcolors)]
            if highlighted_roi is not None and i == highlighted_roi + 1:
                mask_rgb[mask == i] = color
            else:
                mask_rgb[mask == i] = [int(c * opacity) for c in color]
        return cv2.addWeighted(image, 1, mask_rgb, 1, 0)
    
    def drawOutline(self, image, outline, color):
        outline_int = outline.astype(np.int32)
        for i in range(len(outline_int) - 1):
            cv2.line(image, tuple(outline_int[i]), tuple(outline_int[i+1]), color, 2)
        cv2.line(image, tuple(outline_int[-1]), tuple(outline_int[0]), color, 2)
        return image
    
    # Cellpose ROIの透明度更新
    def updateMaskOpacity(self, key, value):
        self.updateStackDisplay(key, self.dict_slider[f"stack_{key}"].value())

    # 指定されたチャンネルの画像を取得する
    def getChannelImage(self, key, channel, index):
        if channel in ["pri", "ref"]:
            image = self.dict_tif_original[channel][index] if key == "fix" else self.dict_tif_reg[channel][index]
            return self.adjustChannelContrast(key, channel, image.copy())
        elif channel == "grid":
            image = self.dict_grid_original[index] if key == "fix" else self.dict_grid_reg[index]
            return image  # 格子画像はコントラスト調整を行わない

    # チャンネルのコントラストを調整する
    def adjustChannelContrast(self, key, channel, image):
        min_val = self.dict_slider[f"minValue_{key}_{channel}"].value()
        max_val = self.dict_slider[f"maxValue_{key}_{channel}"].value()
        return np.clip(((image - min_val) / (max_val - min_val) * 255), 0, 255).astype(np.uint8)

    # RGB画像を作成する
    def createRGBImage(self, key, pri_image, ref_image, grid_image):
        rgb_image = np.zeros((pri_image.shape[0], pri_image.shape[1], 3), dtype=np.uint8)
        rgb_image[:,:,1] = pri_image
        rgb_image[:,:,0] = ref_image
        rgb_image[:,:,2] = grid_image
        return rgb_image

    # 画像を表示する
    def displayImage(self, key, rgb_image):
        height, width, _ = rgb_image.shape
        q_image = QImage(rgb_image.data, width, height, 3 * width, QImage.Format_RGB888)
        pixmap = QPixmap.fromImage(q_image)
        self.dict_scene[key].clear()
        self.dict_scene[key].addPixmap(pixmap)
        self.dict_view[key].setScene(self.dict_scene[key])
        self.dict_view[key].fitInView(self.dict_scene[key].sceneRect(), Qt.KeepAspectRatio)

    # スタックインデックスラベルを更新する
    def updateStackIndexLabel(self, key, index):
        self.dict_label[f"stack_index_{key}"].setText(f"Stack Index: {index}")

    """
    Scene Function
    """
    # CellposeのmaskROI用の色を作成
    def makeCellposeROIColors(self, num_colors):
        colors = []
        for _ in range(num_colors):
            color = [random.randint(0, 255) for _ in range(3)]
            colors.append(color)
        return colors
    
    # Transformationをチェックする格子画像の座標を作る
    def makeGridCoords(self, width=512, height=512, square=32):
        grid_x, grid_y = [], []
        x_, y_ = np.arange(0, width), np.arange(0, height)

        for i in range(len(x_)):
            for j in range(len(y_)):
                if i % square == 0:
                    grid_x += [x_[i]]
                    grid_y += [y_[j]]
                else:
                    if j % square == 0:
                        grid_x += [x_[i]]
                        grid_y += [y_[j]]
        grid = np.array([grid_x, grid_y]).T
        return grid
    # 格子の座標を画像に変換
    def convertGridCoordsToImage(self, grid, width=512, height=512):
        im_mono = np.zeros((height, width), dtype=np.uint8)  # uint8型に変更
        for y, x in grid:
            # 枠外の座標はスキップ
            if 0 <= x < height and 0 <= y < width:
                im_mono[x, y] = 255
        return im_mono
    
    # Viewをクリックしたときの操作
    def viewMousePressEvent(self, event, key):
        if key != "mov" or 'med' not in self.dict_cellpose_stack:
            return

        pos = self.dict_view[key].mapToScene(event.pos())
        click_x, click_y = pos.x(), pos.y()

        stack_index = self.dict_slider[f"stack_{key}"].value()
        meds = self.dict_cellpose_stack['med'][stack_index]

        closest_roi_index = self.findClosestROI(click_x, click_y, meds)

        if closest_roi_index is not None:
            self.selectTableCell(stack_index, closest_roi_index)
            self.updateStackDisplay(key, stack_index)

    def findClosestROI(self, x, y, meds):
        if not meds:
            return None

        distances = [np.sqrt((med[0] - x)**2 + (med[1] - y)**2) for med in meds]
        return distances.index(min(distances))
    
    # 選択したROIのstatを表示
    def updateROIStat(self, stack_index, closest_roi_index):
        cellid = closest_roi_index
        med    = self.dict_cellpose_stack["med"][stack_index][closest_roi_index].astype("int")
        npix   = self.dict_cellpose_stack["npix"][stack_index][closest_roi_index]
        for key, label, value in zip(["roi_cellid", "roi_stat_med", "roi_stat_npix"], ["ROI:", "med:", "npix:"], [cellid, med, npix]):
            self.dict_label[key].setText(f"{label} {value}")
    
    # button widgetに紐づける関数
    """
    Button-binding Function
    """
    # ダイアログを開いて選択したファイルパスをentryに入力
    def openFileDialog(self, fileFilter, entry):
        options = QFileDialog.Options()
        filePath, _ = QFileDialog.getOpenFileName(self, "Open File", "", fileFilter, options=options)
        if filePath:
            entry.setText(filePath)
    
    # 指定したファイルパスを読み込みして初期化
    def loadFilePathsandInitialize(self):
        self.initialize_gui = False
        print("Files Loading...")
        # data読み込み
        self.initializeGUI()

        # UI初期化
        if not self.setupUI_done:
            self.setupUI()
            
        # スタックスライダーの初期化（UIセットアップ後に行う）
        self.initializeStackSliders()
        
        # マッチングテーブルの更新
        self.initializeMatchingTable()

        # 画像の更新
        self.updateStackDisplay("fix", 0)
        self.updateStackDisplay("mov", 0)
        
        print("GUI Initializing...")
        self.initialize_gui = True
        
    # 画像をint型に変換 型は指定可能
    def convertImageToINT(self, im, dtype="uint8"):
        im = im.astype("float")
        im -= np.min(im)
        im /= np.max(im)
        im *= 255
        im = im.astype(dtype)
        return im

            
    """
    ROI Matching Function
    """
    def runROIMatching(self):
        print("ROI matching...")
        if 'med' not in self.dict_cellpose_stack or 'npix' not in self.dict_cellpose_stack:
            QMessageBox.warning(self, "Warning", "Please run Cellpose first.")
            return

        r = float(self.dict_entry["roimatch_threshold_r"].text())
        error_rate = float(self.dict_entry["roimatch_threshold_error_rate"].text().strip('%')) / 100

        model = self.dict_table["mov_match"].model()

        for stack_index in range(len(self.dict_cellpose_stack['med']) - 1):
            curr_meds = self.dict_cellpose_stack['med'][stack_index]
            curr_npixs = self.dict_cellpose_stack['npix'][stack_index]
            next_meds = self.dict_cellpose_stack['med'][stack_index + 1]
            next_npixs = self.dict_cellpose_stack['npix'][stack_index + 1]

            for curr_roi_index, curr_med in enumerate(curr_meds):
                curr_npix = curr_npixs[curr_roi_index]
                best_match = None
                min_distance = float('inf')

                for next_roi_index, next_med in enumerate(next_meds):
                    next_npix = next_npixs[next_roi_index]
                    distance = np.linalg.norm(np.array(curr_med) - np.array(next_med))

                    if distance < r and abs(curr_npix - next_npix) / curr_npix < error_rate:
                        if distance < min_distance:
                            min_distance = distance
                            best_match = next_roi_index

                value = str(best_match) if best_match is not None else "-1"
                model.setData(model.index(curr_roi_index, (stack_index + 1) * 2 - 1), value)

        self.dict_table["mov_match"].setModel(model)
        print("Matching Finished !")
        
    """
    Elastix
    """
    # Elastix Config
    def openElastixConfigWindow(self):
        function = self.dict_combobox["regist_method"].currentText()
        # affine, bsplineを選んでない場合はエラー
        if function == "affine" or function == "bspline":
            self.elastix_config_window = ElastixConfigWindow(self, function)
            self.elastix_config_window.exec_()
        else:
            QMessageBox.information(self, "Elastix Config", "Select 'affine' or 'bspline'!")
            
    # 1枚目のstack画像をfixとし、~枚目のstack画像をmovとしてElastix実行
    def runElastix(self):
        print("Running Registration...")
        function = self.dict_combobox["regist_method"].currentText()

        im_stack_ref = self.dict_tif_original["ref"].copy()
        im_stack_pri = self.dict_tif_original["pri"].copy()
        im_stack_grid = self.dict_grid_original.copy()

        # 1枚目の画像はそのまま保存
        self.dict_tif_reg["ref"][0] = im_stack_ref[0]
        self.dict_tif_reg["pri"][0] = im_stack_pri[0]
        self.dict_grid_reg[0] = im_stack_grid[0]

        for i in range(1, len(im_stack_ref)):
            # Reference channelの登録
            img_fix_ref = itk.image_view_from_array(im_stack_ref[0])
            img_mov_ref = itk.image_view_from_array(im_stack_ref[i])
            img_res_ref, result_transform_parameters = self.elastixRegistrationMethod(img_fix_ref, img_mov_ref, function)
            self.dict_tif_reg["ref"][i] = itk.array_from_image(img_res_ref)

            # Primary channelに同じ変換を適用
            img_fix_pri = itk.image_view_from_array(im_stack_pri[0])
            img_mov_pri = itk.image_view_from_array(im_stack_pri[i])
            img_res_pri = self.applyTransform(img_mov_pri, result_transform_parameters, img_fix_pri)
            self.dict_tif_reg["pri"][i] = itk.array_from_image(img_res_pri)

            # Grid imageに同じ変換を適用
            img_fix_grid = itk.image_view_from_array(im_stack_grid[0])
            img_mov_grid = itk.image_view_from_array(im_stack_grid[i])
            img_res_grid = self.applyTransform(img_mov_grid, result_transform_parameters, img_fix_grid)
            self.dict_grid_reg[i] = itk.array_from_image(img_res_grid)

        print("Registration completed.")
        self.updateStackDisplay("mov", self.dict_slider["stack_mov"].value())
        
    # 画像登録の実行
    def elastixRegistrationMethod(self, img_fix, img_mov, function):
        # ElastixImageFilterの設定
        parameter_object = itk.ParameterObject.New()
        parameter_map = parameter_object.GetDefaultParameterMap(function, 4)
        # Elastix parameter上書き
        dict_parameter_map = self.dict_parameter_map[function]
        for key_parameter, value_parameter in dict_parameter_map.items():
            parameter_map[key_parameter] = value_parameter
        parameter_object.AddParameterMap(parameter_map)

        # エラーの場合はダイアログ表示
        try:
            # 画像登録の実行
            img_res, result_transform_parameters = itk.elastix_registration_method(
                img_fix, img_mov,
                parameter_object=parameter_object)
        except RuntimeError:
            QMessageBox.information(self, "Error", "Runtime Error ! Check Elastix Config !")
        return img_res, result_transform_parameters
    
    # 変換パラメータを適用する関数
    def applyTransform(self, img_mov, transform_parameters, img_fix):
        try:
            img_res = itk.transformix_filter(img_mov, transform_parameters)
            return img_res
        except RuntimeError:
            QMessageBox.information(self, "Error", "Runtime Error ! Failed to apply transform.")
            return img_mov  # エラーが発生した場合、元の画像を返す
        
    """
    Cellpose
    """
    # Cellpose config
    def openCellposeConfigWindow(self):
        self.cellpose_config_window = CellposeConfigWindow(self)
        self.cellpose_config_window.exec_()
        
    # ROIの座標, 中心座標, 面積の抽出
    def extractCellposeROIStatFromMaskStack(self, mask_stack):
        roi_stack, med_stack, npix_stack = [], [], []
        for mask in mask_stack:
            rois, meds, npixs = [], [], []
            for cellid in range(1, len(np.unique(mask))):
                y, x = np.where(mask == cellid)
                roi = np.column_stack((x, y))
                med = np.array((np.mean(x), np.mean(y)))
                npix = len(roi)
                rois.append(roi)
                meds.append(med)
                npixs.append(npix)
            roi_stack.append(rois)
            med_stack.append(meds)
            npix_stack.append(npixs)
        return roi_stack, med_stack, npix_stack
        
    # ROI抽出実行
    def runCellpose(self):
        print("Running Cellpose...")
        model = cellpose.denoise.CellposeDenoiseModel(gpu=self.dict_parameter_cellpose["gpu"], 
                                                      model_type=self.dict_parameter_cellpose["model_type"],
                                                      restore_type=self.dict_parameter_cellpose["restore_type"])
        img_stack = list(self.dict_tif_reg["pri"]) # Cellpose用にlist化
        diam_stack = [self.dict_parameter_cellpose["diameter"]] * len(img_stack)
        mask_stack, flow_stack, style_stack, img_stack_denoise = model.eval(img_stack, 
                                                                            diameter=diam_stack, 
                                                                            channels=self.dict_parameter_cellpose["channels"])
        # 輪郭抽出
        outline_stack = [cellpose.utils.outlines_list(masks=mask) for mask in mask_stack]
        # ROIの座標, 中心座標, 面積の抽出
        roi_stack, med_stack, npix_stack = self.extractCellposeROIStatFromMaskStack(mask_stack)
        
        for key, stack in zip(["img", "mask", "flow", "style", "img_denoise", "outline", "roi", "med", "npix"], 
                              [img_stack, mask_stack, flow_stack, style_stack, img_stack_denoise, outline_stack, roi_stack, med_stack, npix_stack]):
            self.dict_cellpose_stack[key] = stack
            
        # マスクの最大値（ROIの数）を取得し、必要に応じて色のリストを拡張
        max_roi_num = max(mask.max() for mask in self.dict_cellpose_stack['mask'])
        if max_roi_num > len(self.ROIcolors):
            additional_colors = self.makeCellposeROIColors(max_roi_num - len(self.ROIcolors))
            self.ROIcolors.extend(additional_colors)
        
        # Cellpose実行後、movの表示を更新
        self.updateStackDisplay("mov", self.dict_slider["stack_mov"].value())

        # マスク表示を有効にする
        self.dict_checkbox["show_mov_mask"].setChecked(True)

        # ROIの番号をテーブルに割り当てる
        self.assignROIsToTable()
        
        print("Finish Cellpose !")

    def exitApp(self):
        self.close()
        
if __name__ == "__main__":
    app = QApplication(sys.argv) if QApplication.instance() is None else QApplication.instance()
    font = setGUIFont(app)
    gui = MicrogliaTracking()
    gui.show()
    sys.exit(app.exec_())

Files Loading...


In [104]:
bool(gui.dict_table["mov_match"].model().item(1, 0).text())

False

In [72]:
gui.dict_cellpose_stack["med"][0][0].astype("int")

array([748,  16])

In [41]:
gui.dict_cellpose_stack.keys()

dict_keys(['img', 'mask', 'flow', 'style', 'img_denoise', 'outline', 'roi', 'med', 'npix'])

In [45]:
gui.dict_cellpose_stack["npix"][0]

[111,
 126,
 172,
 136,
 152,
 195,
 106,
 121,
 136,
 206,
 220,
 230,
 209,
 220,
 343,
 351,
 320,
 253,
 259,
 274,
 294,
 231,
 253,
 284,
 365,
 363,
 480,
 312,
 427,
 378,
 201,
 181,
 252,
 58,
 287,
 718,
 298,
 552,
 579,
 354,
 459,
 670,
 291,
 204,
 318,
 359,
 191,
 254,
 98,
 648,
 170,
 325,
 584,
 299,
 569,
 296,
 353,
 299,
 446,
 396,
 368,
 614,
 737,
 406,
 403,
 356,
 215,
 502,
 365,
 564,
 350,
 466,
 293,
 433,
 417,
 415,
 342,
 370,
 378,
 268,
 376,
 490,
 330,
 217,
 483,
 276,
 509,
 473,
 574,
 317,
 336,
 425,
 497,
 601,
 213,
 795,
 480,
 343,
 687,
 354,
 198,
 627,
 280,
 605,
 543,
 448,
 173,
 117,
 404,
 129,
 176,
 389,
 657,
 553,
 465,
 267,
 519,
 305,
 499,
 81,
 299,
 187,
 359,
 215,
 211,
 587,
 135,
 235,
 365,
 527,
 237,
 372,
 186,
 203,
 59,
 259,
 272,
 155,
 246,
 360,
 382,
 276,
 199,
 320,
 87,
 147,
 282,
 352,
 245,
 153,
 192,
 528,
 188,
 144,
 126,
 233,
 277,
 291,
 174,
 68,
 119,
 52,
 331,
 383,
 148,
 101,
 123,
 188,

In [30]:
gui.dict_cellpose_stack["outline"][0][0]

array([[746,  10],
       [745,  11],
       [745,  12],
       [744,  13],
       [744,  14],
       [744,  15],
       [744,  16],
       [744,  17],
       [744,  18],
       [745,  19],
       [745,  20],
       [746,  21],
       [746,  22],
       [747,  22],
       [748,  23],
       [749,  24],
       [750,  24],
       [751,  23],
       [752,  23],
       [752,  22],
       [753,  21],
       [753,  20],
       [753,  19],
       [753,  18],
       [753,  17],
       [752,  16],
       [752,  15],
       [752,  14],
       [751,  13],
       [750,  12],
       [750,  11],
       [749,  10],
       [748,  10],
       [747,  10]])

In [21]:
mask = gui.dict_cellpose_stack["mask"][0]

In [37]:
y, x = np.where(mask == 1)
roi = np.column_stack((x, y))
med = np.array((np.mean(x), np.mean(y)))
npix = len(roi)

111

In [45]:
def generate_roi_colors(num_colors):
    """ROIごとの色を生成する"""
    colors = []
    for _ in range(num_colors):
        color = [random.randint(0, 255) for _ in range(3)]
        colors.append(color)
    return colors

generate_roi_colors(999)

[[241, 192, 92],
 [39, 66, 98],
 [118, 40, 204],
 [143, 141, 158],
 [176, 23, 36],
 [239, 228, 128],
 [232, 109, 166],
 [31, 221, 221],
 [0, 67, 63],
 [191, 145, 174],
 [139, 3, 3],
 [137, 207, 253],
 [128, 178, 206],
 [226, 150, 63],
 [183, 69, 60],
 [214, 158, 179],
 [98, 39, 5],
 [157, 140, 212],
 [225, 69, 80],
 [176, 178, 226],
 [140, 133, 113],
 [106, 220, 89],
 [148, 186, 239],
 [37, 122, 40],
 [109, 45, 45],
 [66, 2, 233],
 [62, 85, 16],
 [57, 68, 171],
 [112, 98, 124],
 [237, 110, 157],
 [198, 21, 142],
 [88, 75, 110],
 [32, 58, 211],
 [103, 29, 131],
 [186, 136, 12],
 [194, 113, 119],
 [59, 89, 181],
 [161, 47, 234],
 [34, 252, 155],
 [224, 151, 154],
 [134, 134, 206],
 [16, 32, 92],
 [116, 55, 250],
 [233, 241, 230],
 [184, 14, 115],
 [104, 213, 221],
 [151, 152, 189],
 [149, 186, 73],
 [154, 118, 92],
 [10, 174, 207],
 [107, 114, 155],
 [235, 207, 148],
 [158, 203, 180],
 [204, 155, 12],
 [51, 84, 244],
 [20, 44, 188],
 [234, 164, 196],
 [45, 79, 218],
 [89, 20, 242],
 [65,

In [31]:
gui.dict_parameter_cellpose["channels"]

[0, 0]